In [2]:
import pandas as pd 

In [3]:
# df = pd.read_csv("../data/Batch_5082026_batch_results (1).csv") # prev. versiona
df = pd.read_csv("../data/Batch_5083843_batch_results.csv")

In [4]:
df["Description"].iloc[0]

'Read a 400-word text with medicine-related information and write a 100-word summary.'

In [14]:
def parse_response(x):
    summary = x["summary"]
    actions = eval(x["log_of_what_they_did"])
    
    data = {"keys": "", "dates": []}
    
    for val in actions:
        if (val[0] == "copy") or (val[0] == "paste"):
            data["keys"] += " " + val[0]
            data["dates"].append(None)
        else:
            data["keys"] += " " + val[1]
            data["dates"].append(val[2])
    data["keys"] = " ".join(data["keys"].split()).strip()
    
    return summary, data

In [15]:
# print(df["Answer.taskAnswers"].iloc[5])

In [16]:
relevant_columns = ["HITId", "original_text","key_strokes", "datetime", "summary"]

responses = pd.DataFrame(columns = relevant_columns)

for i, row in df.iterrows():
    unparsed = eval(row["Answer.taskAnswers"])[0]
    summary, data = parse_response(unparsed)
    
    key_strokes = data["keys"]
    datetime = data["dates"]
    
    temp = pd.DataFrame([[row["HITId"], row["Input.texts"], key_strokes, datetime, summary]], columns = relevant_columns)
    
    responses = pd.concat([responses, temp], axis = 0)
    
    

In [18]:
# responses

In [21]:
def locate_paste(x: str) -> bool:
    """
    This function will attempt to use some heuristics to determine
    if the key strokes signify that the text was copied.
    """
    if ("Control v" in x) or ("paste" in x):
        return True
    else:
        return False
    
responses["copied"] = responses["key_strokes"].apply(lambda x: locate_paste(x))

In [22]:
responses.groupby("copied")["HITId"].count()

copied
False     5
True     43
Name: HITId, dtype: int64

In [24]:
responses.to_csv("../data/processed_responses.csv", index=False)

In [25]:
responses[responses["copied"]==False]

,HITId,original_text,key_strokes,datetime,summary,copied
0,3XABXM4AJ1Q8DQKOXVSS6QS7TGU8QF,Effect of Screening Mammography on Breast-Canc...,Shift Shift Shift Shift Shift Shift Shift Shif...,"[2023-06-01T10:42:22.995Z, 2023-06-01T10:42:23...",The Norwegian breast-cancer screening program ...,False
0,3KTCJ4SCVGMZHB2B6HCWA7LU59R1M1,Primary Prevention of Cardiovascular Disease w...,I T W A S Backspace Backspace Backspace Backsp...,"[2023-06-01T07:12:25.109Z, 2023-06-01T07:12:25...",it was a diet plan for a good health and a tri...,False
0,3UY4PIS8QR65ZTNBIIRWNMAUAZ11N1,Waning Protection after Fifth Dose of Acellula...,g o o d,"[2023-06-01T06:37:26.337Z, 2023-06-01T06:37:27...",good,False
0,3UY4PIS8QR65ZTNBIIRWNMAUAZ11N1,Waning Protection after Fifth Dose of Acellula...,copy copy Shift T h i s i s Shift A n Backspac...,"[None, None, 2023-06-01T08:20:30.835Z, 2023-06...",This is an emergency announcement of the child...,False
0,3B623HUYJ4BKQ6Z1PZHX1Q78B4F8S4,"A Population-Based Study of Measles, Mumps, an...",h Backspace c d c r e c o m m e n d s t h Back...,"[2023-06-01T11:58:43.789Z, 2023-06-01T11:58:44...",recommends that people get vaccine to protect...,False


In [83]:
responses[responses["key_strokes"].map(len)> 300]

,HITId,original_text,key_strokes,datetime,summary,copied
0,38Z7YZ2SB3N12WW16H4JJFMLWAMQIO,Comparison of Weight-Loss Diets with Different...,ArrowLeft Backspace ArrowLeft ArrowLeft ArrowL...,"[2023-05-30T02:49:46.401Z, 2023-05-30T02:49:46...",This two-year study found that diets varying i...,True
0,3SBX2M1TKD87UPXMLS2FPEC2RSM4QO,Effect of Screening Mammography on Breast-Canc...,CapsLock A CapsLock s t u d y i n CapsLock N C...,"[2023-05-29T18:40:24.249Z, 2023-05-29T18:40:24...",A study in Norway found that screening mammogr...,False
0,38DCH97KHHNXX7G7OZPJVG1L9PBQJ0,"First Results of Phase 3 Trial of RTS,S/AS01 M...",Backspace Backspace Backspace Backspace Backsp...,"[2023-05-29T18:24:22.677Z, 2023-05-29T18:24:22...",Exciting news from a study on a malaria vaccin...,False
0,38DCH97KHHNXX7G7OZPJVG1L9PBQJ0,"First Results of Phase 3 Trial of RTS,S/AS01 M...",Delete Delete Control v t h e 1 c h i s u s e ...,"[2023-05-29T18:19:27.221Z, 2023-05-29T18:19:27...","A phase 3 study of the malaria vaccine RTS, S/...",True
0,3M93N4X8HK89ANVKG3EOQGGMRDWSJR,Exemestane for Breast-Cancer Prevention in Pos...,Control Control Control Control Control Contro...,"[2023-05-29T18:35:17.459Z, 2023-05-29T18:35:17...",here they discuss about the two members Tamox...,True
0,3087LXLJ6M1T4CPCOB4J1CIB3EX0FA,Effect of Three Decades of Screening Mammograp...,"Shift T o r e d u c e i m m o r t a l i t y , ...","[2023-05-29T20:22:57.964Z, 2023-05-29T20:22:58...","To reduce mortality, cancer screening programs...",False
0,3087LXLJ6M1T4CPCOB4J1CIB3EX0FA,Effect of Three Decades of Screening Mammograp...,b e n i f i t Backspace Backspace Backspace Ba...,"[2023-05-30T04:00:56.353Z, 2023-05-30T04:00:57...",BENIFIT OF MEDICINE\nMedicines are chemicals o...,False
0,3WRBLBQ2GRTZICL7ZVA1JXIN4TX0G9,Primary Prevention of Cardiovascular Disease w...,F11 CapsLock I N A M U L T I C E N T E R T R I...,"[2023-05-30T02:23:03.968Z, 2023-05-30T02:23:17...",IN A MULTICENTER TRIAL IN SPIN WE RANDOMLY ASS...,False
0,3WRBLBQ2GRTZICL7ZVA1JXIN4TX0G9,Primary Prevention of Cardiovascular Disease w...,CapsLock D CapsLock e a r f r i e n d i n t h ...,"[2023-05-30T05:17:38.077Z, 2023-05-30T05:17:38...",In this topic myself tell by friend. And this ...,False
0,3UUSLRKAULOAO3B0I4U2SRPDZFN7D6,Waning Protection after Fifth Dose of Acellula...,Control Control Control Control Control Contro...,"[2023-05-29T20:03:48.474Z, 2023-05-29T20:03:48...",We assessed the risk of pertussis in children ...,True


In [98]:
val = 0

print(responses[responses["HITId"] == "3087LXLJ6M1T4CPCOB4J1CIB3EX0FA"].iloc[val]["original_text"])

print()
print("---")
print()
print(responses[responses["HITId"] == "3087LXLJ6M1T4CPCOB4J1CIB3EX0FA"].iloc[val]["summary"])



Effect of Three Decades of Screening Mammography on Breast-Cancer Incidence

To reduce mortality, screening must detect life-threatening disease at an earlier, more curable stage. Effective cancer-screening programs therefore both increase the incidence of cancer detected at an early stage and decrease the incidence of cancer presenting at a late stage.

We used Surveillance, Epidemiology, and End Results data to examine trends from 1976 through 2008 in the incidence of early-stage breast cancer (ductal carcinoma in situ and localized disease) and late-stage breast cancer (regional and distant disease) among women 40 years of age or older.

The introduction of screening mammography in the United States has been associated with a doubling in the number of cases of early-stage breast cancer that are detected each year, from 112 to 234 cases per 100,000 women - an absolute increase of 122 cases per 100,000 women. Concomitantly, the rate at which women present with late-stage cancer has de

In [99]:
responses["key_strokes"].iloc[8]

'Control v'

'Sibutramine is a weight loss drug that has been shown to be effective in reducing body weight in overweight and obese adults. However, the long-term effects of sibutramine on cardiovascular health have not been well studied.\n\nIn this study, researchers conducted a randomized controlled trial to assess the cardiovascular effects of sibutramine in overweight and obese adults with preexisting cardiovascular disease, type 2 diabetes mellitus, or both. The study included 10,744 participants who were randomly assigned to receive either sibutramine or placebo for 3.4 years.\n\nThe results of the study showed that sibutramine was associated with an increased risk of nonfatal myocardial infarction and nonfatal stroke. However, sibutramine was not associated with an increased risk of cardiovascular death or death from any cause.\n\nBased on the results of this study, the researchers concluded that sibutramine should not be used in overweight and obese adults with preexisting cardiovascular di